# Predicting heart disease using machine learning

This notebook looks into various Python based data science and machine learning libraries to predict whether a person has heart disease or not.

We are going to take following approach.
1. Problem definition
2. Data
3. Evaluation
4. Features
5. Modeling
6. Experimentation

## 1. Problem definition
> Given clinical parameters about a patient, we need to identify whether a person has heart disease or not.

## 2. Data
Original Data
https://archive.ics.uci.edu/ml/datasets/Heart+Disease

Kaggle 
https://www.kaggle.com/ronitf/notebook


## 3. Evaluation
> If the accuracy is achieved 95% to predict if a person has heart disease or not, we will pursue the project.

## 4. Features

Only 14 attributes used:
1. #3 (age)
2. #4 (sex)
3. #9 (cp)
4. #10 (trestbps)
5. #12 (chol)
6. #16 (fbs)
7. #19 (restecg)
8. #32 (thalach)
9. #38 (exang)
10. #40 (oldpeak)
11. #41 (slope)
12. #44 (ca)
13. #51 (thal)
14. #58 (num) (the predicted attribute)

## Preparing Tools

Importing all the tools that we need

In [3]:
# Regualr EDA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Models from Scikit Learn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Model Evaluations
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import plot_roc_curve